In [1]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/santander_2016-master

/content/drive/My Drive/santander_2016-master


In [3]:
!pip install mlxtend

In [ ]:
!pip install xgboost

In [5]:
import numpy as np
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict

In [4]:
!pip install --upgrade lightgbm

     |████████████████████████████████| 2.0 MB 5.8 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [8]:
import lightgbm as lgbm

Đây là sử dụng mô hình LightGBM, chạy trên cách tiền xử lý và dự đoán mô hình của Marios

In [ ]:
def loadcolumn(filename,col=4, skip=1, floats=True):
    pred=[]
    op=open(filename,'r')
    if skip==1:
        op.readline() #header
    for line in op:
        line=line.replace('\n','')
        sps=line.split(',')
        #load always the last columns
        if floats:
            pred.append(float(sps[col]))
        else :
            pred.append(str(sps[col]))
    op.close()
    return np.array(pred)            

    
def load_datas(filename):

    return joblib.load(filename)

def printfile(X, filename):

    joblib.dump((X), filename)
    
def printfilcsve(X, filename, headers):

    np.savetxt(filename,X, header=headers) 
    
    
def load_ids(id_file, cols=20):
    verybiglist=[]
    for s in range(0,cols):
        idss=loadcolumn(id_file,col=s, skip=1, floats=True)
        id_list=[ [] ,[] , [], [] , []]
        id_dict=[ defaultdict(int) ,defaultdict(int) , defaultdict(int), defaultdict(int) , defaultdict(int)]
        for g in range(0,len(idss)):
            id_list[int(idss[g])].append(g)
            id_dict[int(idss[g])][g]=1
        biglist=[]
        for k in range(5):
            training_ids=[s for s in range(0,len(idss)) if s not in id_dict[k] ]
            biglist.append([training_ids,id_list[k] ])
            print(len(biglist), len(biglist[0]))
        verybiglist.append(biglist)
            
    return verybiglist

In [ ]:
def all_load_vecorizerr(tr,te,drop=["ind_var2_0","ind_var2","ind_var27_0","ind_var28_0","ind_var28","ind_var27",
"ind_var41","ind_var46_0","ind_var46","num_var27_0","num_var28_0","num_var28","num_var27","num_var41","num_var46_0",
"num_var46","saldo_var28","saldo_var27","saldo_var41","saldo_var46","imp_amort_var18_hace3","imp_amort_var34_hace3",
"imp_reemb_var13_hace3","imp_reemb_var33_hace3","imp_trasp_var17_out_hace3","imp_trasp_var33_out_hace3",
"num_var2_0_ult1","num_var2_ult1","num_reemb_var13_hace3","num_reemb_var33_hace3","num_trasp_var17_out_hace3",
"num_trasp_var33_out_hace3","saldo_var2_ult1","saldo_medio_var13_medio_hace3","ind_var6_0","ind_var6",
"ind_var13_medio_0","ind_var18_0","ind_var26_0","ind_var25_0","ind_var32_0","ind_var34_0","ind_var37_0",
"ind_var40","num_var6_0","num_var6","num_var13_medio_0","num_var18_0","num_var26_0","num_var25_0","num_var32_0",
"num_var34_0","num_var37_0","num_var40","saldo_var6","saldo_var13_medio","delta_imp_reemb_var13_1y3",
"delta_imp_reemb_var17_1y3","delta_imp_reemb_var33_1y3","delta_imp_trasp_var17_in_1y3","delta_imp_trasp_var17_out_1y3",
"delta_imp_trasp_var33_in_1y3","delta_imp_trasp_var33_out_1y3"]):

    train  = pd.read_csv(tr, sep=',',quotechar='"')
    test  = pd.read_csv(te, sep=',',quotechar='"')
    train.drop('ID', axis=1, inplace=True)
    train.drop('TARGET', axis=1, inplace=True)    
    test.drop('ID', axis=1, inplace=True)
    for name in drop:
        train.drop(name, axis=1, inplace=True)    
        test.drop(name, axis=1, inplace=True)        

    train['zerocount'] = train.apply(lambda x: np.sum(x == 0), axis=1)
    test['zerocount'] = test.apply(lambda x: np.sum(x == 0), axis=1)

    train['var38'].replace(117310.979016494, -1.0, inplace=True)
    test ['var38'].replace(117310.979016494, -1.0, inplace=True)
    
    train_s = train
    test_s = test
    result = pd.concat([test_s,train_s])
    
    #test_s.drop('id', axis=1, inplace=True)
    result=result.T.to_dict().values()
    train = train_s.T.to_dict().values()
    test = test_s.T.to_dict().values()
    
    vec = DictVectorizer()
    vec.fit(result)
    train = vec.transform(train)
    test = vec.transform(test)
    
    print(train.shape)
    print(test.shape)    
    
    
    return train,test

In [ ]:
def bagged_set(X_t,y_c,model, seed, estimators, xt, update_seed=True):
    
   # create array object to hold predictions 
   baggedpred=[ 0.0  for d in range(0, (xt.shape[0]))]
    
   for n in range (0, estimators):
          
        if update_seed:
            model.set_params(random_state=seed + n)
        model.fit(X_t,y_c)
        preds=model.predict_proba(xt)[:,1]
       
        for j in range (0, (xt.shape[0])):           
                baggedpred[j]+=preds[j]
                
   for j in range (0, len(baggedpred)): 
                baggedpred[j]/=float(estimators)
             
   return np.array(baggedpred) 

In [10]:
def main():
    
        load_data=True      
        metafolder_train="data/output/train/"
        metafolder_test="data/output/test/"        
        input_folder="data/input/"
        feature_folder="data/output/features/"
        SEED=15
        outset="lightgbm_1" # predic of all files
        number_of_folds=5 # repeat the CV procedure 10 times to get more precise results       
        
        ######### Load files ############

        y=loadcolumn(input_folder+ "train.csv",col=370, skip=1, floats=True)
        ids=loadcolumn(input_folder+ "test.csv",col=0, skip=1, floats=True)
        idstrain=loadcolumn(input_folder+ "train.csv",col=0, skip=1, floats=True)
        keepfold=[0 for k in range(len(y))]
        
        if load_data:
            X,X_test=all_load_vecorizerr(input_folder+'train.csv',input_folder+'test.csv') 
            printfile(X,"Xvector.pkl")  
            printfile(X_test,"Xtestvector.pkl")                               
            X=load_datas("Xvector.pkl").toarray()
            X_test=load_datas("Xtestvector.pkl").toarray()


        tsn_features=(np.loadtxt(feature_folder+ "tsne_feats.csv", delimiter=",", skiprows=1, usecols=[1,2]))
        
        tsn_features_train=tsn_features[:X.shape[0]]
        tsn_features_test=tsn_features[X.shape[0]:tsn_features.shape[0]]     
        
        print(tsn_features_train.shape)
        print(tsn_features_test.shape)

        X=np.column_stack((X,tsn_features_train))     
        X_test=np.column_stack((X_test,tsn_features_test)) 

        print(X.shape)       
        print(X_test.shape)

        kmeans_feats=(np.loadtxt(feature_folder+"kmeans_feats.csv", delimiter=",", skiprows=1, usecols=[1,2,3,4,5,6,7,8,9]))
        
        kmeans_feats_train=kmeans_feats[:X.shape[0]]
        kmeans_feats_test=kmeans_feats[X.shape[0]:tsn_features.shape[0]]     
        
        print(kmeans_feats_train.shape)
        print(kmeans_feats_test.shape)

        X=np.column_stack((X,kmeans_feats_train))     
        X_test=np.column_stack((X_test,kmeans_feats_test)) 

        print(X.shape)
        print(X_test.shape)      


        #model to use
                                                           
        model=lgbm.LGBMClassifier(n_estimators = 1000, num_leaves = 35, max_depth = 5, learning_rate = 0.001,
                                  objective = 'binary', min_child_weight = 1, min_child_samples = 30,
                                  subsample = 0.9, colsample_bytree = 0.45, random_state = 1) 
        #Create Arrays for meta
        train_stacker=[ 0.0  for k in range (0,(X.shape[0])) ]
        test_stacker=[0.0  for k in range (0,(X_test.shape[0]))]
        
        # CHECK EVerything in five..it could be more efficient     
        
        #create target variable        
        print("kfolder")
        
        #load the 20-fold ids.
        kfolders=load_ids(input_folder+"5fold_20times.csv")  
        
        printfile(kfolders,"kfolder.pkl")    
        
        fcount=0
        for kfolder in kfolders:
            mean_kapa = 0.0
            i=0 # iterator counter
            print("starting cross validation with %d kfolds " % (number_of_folds))
            if number_of_folds>0:
                for train_index, test_index in kfolder:
                    # creaning and validation sets
                    X_train, X_cv = X[train_index], X[test_index]
                    y_train, y_cv = np.array(y)[train_index], np.array(y)[test_index]
                           
                    print("folder %d  train size: %d. test size: %d, cols: %d " % (fcount, (X_train.shape[0]) ,(X_cv.shape[0]) ,(X_train.shape[1]) ))
    
                    
                    preds=bagged_set(X_train,y_train,model, SEED, 3, X_cv, update_seed=True)        
                    # compute Loglikelihood metric for this CV fold
                    #scalepreds(preds)     
                    kapa = roc_auc_score(y_cv,preds)
                    print("folder %d size train: %d size cv: %d AUC (fold %d/%d): %f" % (fcount,(X_train.shape[0]), (X_cv.shape[0]), i + 1, number_of_folds, kapa))
                    mean_kapa += kapa
                    #save the results
                    no=0
                    for real_index in test_index:
                             train_stacker[real_index]+=(preds[no])
                             keepfold[real_index]=i
                             no+=1
                    i+=1
            fcount+=1
            print("==============================================================================================")
        for u in range(0,len(train_stacker)):
            train_stacker[u]/=float(len(kfolders))
        grand_auc=roc_auc_score(y, train_stacker)
        print(" Grand AUC: %f" % (grand_auc) )
        if (number_of_folds)>0:
            mean_kapa/=number_of_folds
            print(" printing train datasets ")
            printfilcsve(np.column_stack((np.array(idstrain),np.array(train_stacker))), metafolder_train+ outset  + ".train.csv","ID,TARGET")  
 
        print(" making test predictions ")        
        preds=bagged_set(X, y,model, SEED, 5, X_test, update_seed=True) 
         
        for pr in range (0,len(preds)):            
                    test_stacker[pr]=(preds[pr]) 
    
        preds=np.array(preds)
        printfilcsve(np.column_stack((np.array(ids),np.array(test_stacker))),  metafolder_test+ outset  + ".test.csv","ID,TARGET")                

if __name__=="__main__":
  main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: UserWarning: DataFrame columns are not unique, some columns will be omitted.


(76020, 307)
(75818, 307)
(76020, 2)
(75818, 2)
(76020, 309)
(75818, 309)
(76020, 9)
(75818, 9)
(76020, 318)
(75818, 318)
kfolder
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
1 2
2 2
3 2
4 2
5 2
starting cross validation with 5 kfolds 
folder 0  train size: 60790. test size: 15230, cols: 318 
folder 0 size train: 60790 size cv: 15230 AUC (fold 1/5): 0.828036
folder 0  train size: 60879. test size: 15141, cols: 318 
folder 0 size train: 60879 size cv: 15141 AUC (fold 2/5): 0.824363
folder 0  train size: 60739. test size: 15281, cols: 318 
folder 0 size train: 60739 size cv: 15281 AUC (fold 3/5): 0.829903
folder 0  train size: 60752. test size: 15268, cols: 3